<a href="https://colab.research.google.com/github/selvakannanjr/text_sentiment_analysis/blob/master/Python_project-19PD31%2C19PD33.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer 
from keras.utils import to_categorical

tf=pd.read_csv("/content/prodata.csv")

X_train,X_test,y_train,y_test=train_test_split(tf['content'].values,tf['sentiment'].values,random_state=0)

vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)

from keras.models import Sequential
from keras import layers

intdim=train_vectors.shape[1]
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)


model=Sequential()
model.add(layers.Dense(10,input_dim=intdim,activation='relu'))
model.add(layers.Dense(3,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

history=model.fit(train_vectors,y_train,epochs=100,verbose=False,validation_data=(test_vectors,y_test),batch_size=20)

loss,acc=model.evaluate(test_vectors,y_test,verbose=False)

print(acc)






Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                348630    
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 33        
Total params: 348,663
Trainable params: 348,663
Non-trainable params: 0
_________________________________________________________________
0.4593999981880188


In [25]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#nltk.download('punkt')
#nltk.download('stopwords') 
#nltk.download('wordnet')
from nltk.corpus import stopwords
import re 
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import json

train=pd.read_csv("/content/train.tsv",sep='\t')
test=pd.read_csv("/content/test.tsv",sep='\t')

lem=WordNetLemmatizer()
def clean_text(frame):
    processed=[]
    for sentence in tqdm(frame['Phrase']):
      sentence=sentence.lower()                 # Converting to lowercase
      cleanr=re.compile('<.*?>@')
      sentence=re.sub(cleanr, ' ', sentence)        #Removing HTML tags
      sentence=re.sub(r'[?|!|\'|"|#]',r'',sentence)
      sentence=re.sub(r'[.|,|)|(|\|/]',r' ',sentence)        #Removing Punctuations
      words=[lem.lemmatize(word) for word in sentence.split() if word not in stopwords.words('english')]
      processed.append(words)
    return processed

#trs=clean_text(train)
#tes=clean_text(test)

#with open('/content/trainlist.txt','w') as filehandle:
 # json.dump(trs,filehandle)
#with open('/content/testlist.txt','w') as filehandle:
 # json.dump(tes,filehandle)

with open('/content/trainlist.txt','r') as filehandle:
  trs=json.load(filehandle)
with open('/content/testlist.txt','r') as filehandle:
  tes=json.load(filehandle)

#so far we've processed and loaded the data. now begins the prep for DL

from keras.utils import to_categorical #to convert the target labels to OneHotEncoded form
import random
#from tensorflow import set_random_seed
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence #convert the tokens into sequences in order to be fed to the lstm
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.callbacks import EarlyStopping #save time by terminating the learning when the model stops improving
from keras.losses import categorical_crossentropy #loss function suitable for multi-label classification
from keras.optimizers import Adam
from keras.models import Sequential
from keras.models import load_model

#set_random_seed(343)
random.seed(343) #common seed for all pseudorandom integers in the following code

tar=train.Sentiment.values
y_target=to_categorical(tar)
no_of_labels=y_target.shape[1]

X_train,X_val,y_train,y_val=train_test_split(trs,y_target,random_state=0,test_size=0.2,stratify=y_target)
unique_words = set()
len_max = 0


for sent in X_train:
    
    unique_words.update(sent)
    
    if(len_max<len(sent)):
        len_max = len(sent)

nw=len(list(unique_words))

tokenizer=Tokenizer(num_words=nw)
tokenizer.fit_on_texts(list(X_train))   #now the 'nw' no of most common words will be on the front of the list

X_train=tokenizer.texts_to_sequences(X_train)           #most common words are convtd to sequences 
X_train=sequence.pad_sequences(X_train, maxlen=len_max) #since not all content are of same len,we pad those that are small with zeroes

X_val=tokenizer.texts_to_sequences(X_val)
X_val=sequence.pad_sequences(X_val, maxlen=len_max)

X_test=tokenizer.texts_to_sequences(tes)
X_test=sequence.pad_sequences(X_test, maxlen=len_max)

early_stopping=EarlyStopping(min_delta=0.01,mode='max',monitor='val_accuracy',patience=3)
callback=[early_stopping]
'''
model=Sequential()
model.add(Embedding(nw,300,input_length=len_max))
model.add(LSTM(128,dropout=0.5,recurrent_dropout=0.5,return_sequences=True))
model.add(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='relu'))
model.add(Dense(no_of_labels,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.05),metrics=['accuracy'])
model.summary()

history=model.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=6,batch_size=256,verbose=1,callbacks=callback)

model.save("/content/model.h5")
'''
model=load_model('model.h5')
model.summary()
ynew=model.predict_proba(X_test)

for i in range(10,21):
  print("X=%s, Predicted=%s" % (X_test[i], ynew[i]))
  
    

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 30, 300)           4494900   
_________________________________________________________________
lstm_5 (LSTM)                (None, 30, 128)           219648    
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               6500      
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1010      
_________________________________________________________________
dense_8 (Dense)              (None, 5)                